# CSCM72 Coursework 1: Python Optimisers

Name: Neil Woodhouse<br>
Student Number: 851182

# Section 1: Function Implementation & Testing

In [5]:
import numpy as np
import math
from scipy.stats import truncnorm

In [6]:
# Constants to improve readability
HEIGHT = 0     #x1
LENGTH = 1     #x2
THICKNESS = 2  #x3
BREADTH = 3    #x4

# Store the default upper and lower bound for x1..x4 as defined in the specification
DEFAULT_LOWER = np.array([0.125, 0.125, 0.1, 0.1])
DEFAULT_UPPER = np.array([5.0, 5.0, 10.0, 10.0])

## Constraint Function Execution Tracking
The number of executions of each constraint function is tracked with a global variable. 
A function is included that prints the state of all these variables, to make checking them easier. 

In [7]:
# Counter Variables to track the number of time Constraint Functions have been executed
f_counter = 0
g1_counter = 0
g2_counter = 0
g3_counter = 0
g4_counter = 0

# Prints how many times each constraint function has been executed
def print_execution_trackers():
    global f_counter
    global g1_counter
    global g2_counter
    global g3_counter
    global g4_counter
    print(
        "Constraint function Execution counts: \nf: {}, g1: {}, g2: {}, g3: {}, g4: {}".format(f_counter, g1_counter, g2_counter, g3_counter, g4_counter)
    )

# Function to reset constraint function execution counters (unused, retained to make testing easier)
def reset_execution_trackers():
    global f_counter
    f_counter = 0
    global g1_counter 
    g1_counter = 0
    global g2_counter 
    g2_counter = 0
    global g3_counter 
    g3_counter = 0
    global g4_counter 
    g4_counter = 0

## Function Implementations

This subsection defines the function implementations for the objective function and each of the constraint functions. 

### Objective Function *f(x)*:

In [8]:
def objective_calc(design):
    global f_counter
    f_counter += 1
    return ( 1.10471 * (design[HEIGHT] ** 2) * design[LENGTH]) + (0.04811 * design[THICKNESS] * design[BREADTH] * (14.0 + design[LENGTH]) )

### Shear Stress *g<sub>1</sub>(x)*:

In [9]:
def shear_stress(design):
    # Calculates the first derivative
    tau1 = 6000 / (math.sqrt(2) * design[HEIGHT] * design[LENGTH])

    # Calculates the second derivative
    tau2 = (6000 * (14 + 0.5 * design[LENGTH]) * math.sqrt(0.25 * (design[LENGTH]**2 + (design[HEIGHT] + design[THICKNESS])**2 ) ) )  \
                        / ( 2 * (0.707 * design[HEIGHT] * design[LENGTH] * ( ( (design[LENGTH] ** 2) / 12) + 0.25 * (design[HEIGHT] + design[THICKNESS])**2 ) ) )
    
    # Calculates the overall shear stress of the design
    tau = math.sqrt( (tau1**2 + tau2**2) + ( (design[LENGTH] * tau1 * tau2) \
           / (math.sqrt(0.25*(design[LENGTH]**2 + (design[HEIGHT] + design[THICKNESS])**2))) ))

    global g1_counter
    g1_counter += 1
    return 13600 - tau

### Normal Stress *g<sub>2</sub>(x)*:

In [10]:
def normal_stress(design):
    # Calculates normal stress of the design
    sigma = 504000 / (design[THICKNESS]**2 * design[BREADTH])

    global g2_counter
    g2_counter += 1
    return (30000 - sigma)

### Practicality *g<sub>3</sub>(x)*:

In [11]:
def practicality(design):
    global g3_counter
    g3_counter += 1
    return (design[BREADTH] - design[HEIGHT])

### Buckling Load *g<sub>4</sub>(x)*:

In [12]:
def buckling_load(design):
    rho = 64746.022 * (1 - 0.0282346 * design[THICKNESS]) * design[THICKNESS] * design[BREADTH]**3

    global g4_counter
    g4_counter += 1
    return (rho - 6000)

## Function Validation:

This section checks the validity of the constraint functions, using the example input design from the specification.

In [13]:
x = np.array([1.05, 3.15, 4.43, 7.87])

print("Objective Function Output: ", objective_calc(x))
print("First constraint function output: ", shear_stress(x))
print("Second constraint function output: ", normal_stress(x))
print("Third constraint function output: ", practicality(x))
print("Fourth constraint function output: ", buckling_load(x))

Objective Function Output:  32.6024179859
First constraint function output:  5308.848564674312
Second constraint function output:  26736.764990548952
Third constraint function output:  6.82
Fourth constraint function output:  122317448.61430933


In [14]:
print_execution_trackers()

Constraint function Execution counts: 
f: 1, g1: 1, g2: 1, g3: 1, g4: 1


# Section 2: Random Search

This code defines a flexible random search that takes a system of any 4 parameters and 4 constraint functions and returns a system design that approximately minimises the objective value.

In [15]:
# This function performs random search optimisation using a given seed for number generation, and a given number of samples
# Defaults to minimising the given objective function, but this can be changed to maximising by setting 'minimising' to false
def random_search(seed, samples, lower, upper, objective, constraint1, constraint2, constraint3, constraint4, minimising=True):
    # Initialises algorithm variables
    best_design = None
    objective_minimum = float('inf')
    objective_maximum = float('-inf')

    # Creates local random generator to isolate from seed changes in any other code
    rng = np.random.default_rng(seed)

    for i in range(samples):
        curr_design = np.array([
            rng.uniform(lower[0], upper[0]),
            rng.uniform(lower[1], upper[1]),
            rng.uniform(lower[2], upper[2]),
            rng.uniform(lower[3], upper[3])
            ])

        # Combining the constraints in a single evalution reduces unnecessary execution, since after one constraint function returns false, the rest will not execute
        if (constraint1(curr_design) >= 0) and (constraint2(curr_design) >= 0) and (constraint3(curr_design) >= 0) and (constraint4(curr_design) >= 0):
            curr_result = objective(curr_design)
            if minimising and curr_result < objective_minimum:
                objective_minimum = curr_result
                best_design = curr_design
            if not minimising and curr_result > objective_maximum:
                objective_maximum = curr_result
                best_design = curr_design
    if minimising:
        return best_design, objective_minimum
    else:
        return best_design, objective_maximum

The following section provides a wrapper function that runs a random search with the parameters from the specification.

In [16]:
def default_random_search(seed, samples):
    return random_search(seed, samples, DEFAULT_LOWER, DEFAULT_UPPER, objective_calc, shear_stress, normal_stress, practicality, buckling_load)


### Test Code: Unused, commented out

The following code does a few test runs of random search to ensure it works as intended. <br>
It uses the execution tracker reset function, so has been commented out

In [29]:
# Test Code for random search implementation
"""
SEED = 18002
reset_execution_trackers()
print("Best Design: {0[0]}, with Value = {0[1]}".format(random_search(SEED, 10000, DEFAULT_LOWER, DEFAULT_UPPER, objective_calc, shear_stress, normal_stress, practicality, buckling_load, False)))
print_execution_trackers()
reset_execution_trackers()
print("Best Design: {0[0]}, with Value = {0[1]}".format(default_random_search(SEED, 10000)))
print_execution_trackers()
reset_execution_trackers()
print("Best Design: {0[0]}, with Value = {0[1]}".format(default_random_search(SEED, 10)))
print_execution_trackers()
"""

Best Design: [4.91219271 4.93595545 9.41231517 9.31503612], with Value = 211.4477300135287
Constraint function Execution counts: 
f: 4841, g1: 10000, g2: 7838, g3: 6353, g4: 4841
Best Design: [0.52367905 2.67381647 7.43394281 0.56708869], with Value = 4.191789951308734
Constraint function Execution counts: 
f: 4841, g1: 10000, g2: 7838, g3: 6353, g4: 4841
Best Design: [0.74232398 4.16008388 3.78729206 2.72526361], with Value = 11.550020182303808
Constraint function Execution counts: 
f: 4, g1: 10, g2: 9, g3: 5, g4: 4


# Section 3: Simulated Annealing

This section defines an implementation of simulated annealing, along with fast decay, logarithmic decay, and truncated gaussian functions to use with it. 

In [18]:
def fast_decay(initial_temp, time):
    return initial_temp / time

def log_decay(initial_temp, time):
    return initial_temp * (math.log(2) / math.log(time + 1))

In [70]:


def truncated_gaussian(seed, design, lower, upper, step_size):
    rng = np.random.default_rng(seed)
    new_design = np.array([None,None,None,None])

    for i in range(len(design)):
        # Selects a random, appropriately bounded, value for the current parameter
        new_design[i] = truncnorm.rvs( 
            (lower[i] - design[i]) / step_size,
            (upper[i] - design[i] / step_size),
            loc = design[i],
            scale = step_size,
            random_state = rng
            )
    return new_design
        

In [71]:
def simulated_annealing(seed, samples, lower, upper, objective, constraint1, constraint2, constraint3, constraint4, step_size, initial_temp, decay=fast_decay, execution_limit = None):
    rng = np.random.default_rng(seed)
    execution_count = 0

    # This section ensures the initial starting design is valid under the constraint functions
    valid_design = False
    while not valid_design:
        # This check ensures the code does not exceed the execution limits specified for function evaluations
        if execution_limit is not None and execution_count >= execution_limit:
            break
        initial_design = np.array([
                rng.uniform(lower[0], upper[0]),
                rng.uniform(lower[1], upper[1]),
                rng.uniform(lower[2], upper[2]),
                rng.uniform(lower[3], upper[3])
                ])
        if (constraint1(initial_design) >= 0) and (constraint2(initial_design) >= 0) and (constraint3(initial_design) >= 0) and (constraint4(initial_design) >= 0):
            best_design = initial_design
            valid_design = True
        execution_count += 1

    objective_min = objective(best_design)

    curr_design = best_design
    curr_objective = objective(curr_design)

    for i in range(1, samples):
        if execution_limit is not None and execution_count >= execution_limit:
            break
        new_design = truncated_gaussian(seed, curr_design, lower, upper, step_size)
        new_output = objective(new_design)
        objective_change = new_output - curr_objective
        if (constraint1(curr_design) >= 0) and (constraint2(curr_design) >= 0) and (constraint3(curr_design) >= 0) and (constraint4(curr_design) >= 0):
            if objective_change <= 0 or rng.uniform(0,1) < min(math.exp(-objective_change/decay(initial_temp, i+1)), 1):
                curr_design = new_design
                curr_objective = new_output
            if new_output < objective_min:
                best_design = new_design
                objective_min = new_output
        execution_count += 1
    return best_design, objective_min

### Test Code - Unused, commented out

The following code does a test run of the simulated annealing implementation. <br>
It uses the execution tracker reset function, so has been commented out.

In [76]:
# This section is some test code for the simulated annealing implementation
"""
SEED = 18002
reset_execution_trackers()
print("Best Design: {0[0]}, with Value = {0[1]}".format(
    simulated_annealing(SEED, 10000, DEFAULT_LOWER, DEFAULT_UPPER, objective_calc, shear_stress, normal_stress, practicality, buckling_load, 0.1, 10, fast_decay)
    ))
print_execution_trackers()
reset_execution_trackers()
"""

Best Design: [0.4907532580386743 0.4766058343340802 0.9926020719381352
 0.9989484049257795], with Value = 0.8173944019555847
Constraint function Execution counts: 
f: 10001, g1: 10001, g2: 2, g3: 2, g4: 2


# Section 4: Optimiser Evaluation

This section performs 21 runs of both Random, and Simulated Annealing optimisers, then graphs and evaluates their performance. 